# Pre calculation of Gaunt coefficients

This notebook is used to pre-calculate Gaunt coefficients that are used in the
ViPErLEED Delta-Amplitude calculation. These coefficients are somewhat expensive
to calculate, but can be easily tabulated.

Note, the Gaunt coefficients (also referred to as Wigner 3-j symbols) are
directly related to the Clebsch-Gordan coefficients via mathematical relations
(see also https://en.wikipedia.org/wiki/3-j_symbol). The terms are used
interchangeably in LEED literature.

We use the Sympy physics.wigner module to calculate the values and store them in
a numpy .npy file. 

@authors: @amimre and @Paulhai7

In [ ]:
import numpy as np
from sympy.physics.wigner import gaunt
from pathlib import Path

In [ ]:
# Maximum angular momentum
LMAX = 18

We can make use of selection rules for the Gaunt coefficients. Non-zero values
only occur for:

1) $ M1 + M2 + M3 = 0 $
2) $ |L1-L2| <= L3 <= L1+L2 $

The first condition means that we can skip the loop over M3, as only 
$M3 = -M1-M2$ will yield non-zero values. We can skip this dimension entirely
which reduces the memory requirements and file size siginificantly.
The second condition means we can reduce the range for each $L3$
from $(0,2*LMAX+1)$ to $(|L1-L2|, L1+L2)$.

We employ a compressed storage scheme based on DOI:10.1137/S1064827503422932.

This reduces the file size of the stored coefficients by about a factor of ~5
for LMAX=18.

In [ ]:
# First we generate the index mapping for the storage scheme
index_dict = {}
LMAX = 18
for l1 in range(LMAX+1):
    for l2 in range(LMAX+1):
        for l3 in range(abs(l1-l2), l1+l2+1):
            for m1 in range(-l1, l1+1):
                for m2 in range(-l2, l2+1):
                    m3 = -m1-m2
                    if abs(m3)<=l3:
                        l_unsorted = np.array([l1,l2,l3])
                        m_unsorted = np.array([m1,m2,m3])
                        sorted_indices = np.argsort(l_unsorted)[::-1]
                        l = l_unsorted[sorted_indices]
                        m = m_unsorted[sorted_indices]
                        if m[2] < 0:
                            m = -m
                        index = l[0]*(6+l[0]*(11+l[0]*(6+l[0])))/24 + l[1]*(2+l[1]*(3+l[1]))/6 + l[2]*(l[2]+1)/2 + m[2] + 1
                        if index not in index_dict:
                            index_dict[index] = abs(m[1])
                        else:
                            if index_dict[index] < abs(m[1]):
                                index_dict[index] = abs(m[1])

index_array = []
for key, value in index_dict.items():
    index_array.append([int(key),value])
index_array = np.array(index_array).T
sort_indices = np.argsort(index_array[0])
index_array[0] = index_array[0][sort_indices]
index_array[1] = index_array[1][sort_indices]

Now we calculate all required Gaunt coefficients, skipping those we don't need.
Note this calculation will take ~5 minutes for LMAX=18.

In [ ]:
compressed_gaunt = np.full(shape=(index_array[0,-1]+1,max(index_array[1]*2+1)),dtype=np.float64,fill_value=np.nan)
compressed_gaunt[0,0] = 0.0
for l1 in range(LMAX+1):
    for l2 in range(LMAX+1):
        for l3 in range(abs(l1-l2), l1+l2+1):
            for m1 in range(-l1, l1+1):
                for m2 in range(-l2, l2+1):
                    m3 = -m1-m2
                    if abs(m3)<=l3:
                        l_unsorted = np.array([l1,l2,l3])
                        m_unsorted = np.array([m1,m2,m3])
                        sorted_indices = np.argsort(l_unsorted)[::-1]
                        l = l_unsorted[sorted_indices]
                        m = m_unsorted[sorted_indices]
                        index = l[0]*(6+l[0]*(11+l[0]*(6+l[0])))/24 + l[1]*(2+l[1]*(3+l[1]))/6 + l[2]*(l[2]+1)/2 + abs(m[2]) + 1
                        index = int(index)
                        if m[2]>=0:
                            my_m = m[1]
                        else:
                            my_m = -m[1]
                        index2 = (len(compressed_gaunt[index,:])-1)/2 + my_m
                        index2 = int(index2)
                        if np.isnan(compressed_gaunt[index,index2]):
                            compressed_gaunt[index,index2] = float(gaunt(l[0], l[1], l[2], m[0], m[1], m[2]).doit())


In [ ]:
# Size in memory
print(round(compressed_gaunt.__sizeof__() / 1024**2, 2), "MB")

In [ ]:
# Save to file
file = Path('.') / 'gaunt_coefficients.npy'
np.save(file, compressed_gaunt)

In [ ]:
# Size on disk
print(round(file.stat().st_size / 1024**2, 2), "MB")

In [ ]:
# Load from file and check that it is the same
loaded = np.load(file)
print(np.allclose(compressed_gaunt, loaded, equal_nan=True))

In [ ]:
loaded.dtype